In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_excel('data/conference_board_data.xlsx', engine='openpyxl',header=None)

In [6]:
# Declare parameters
repeated_rows = []
empty_rows = []

''' 
Loop to identify which rows contain one value or blanks. 
@param empty_rows -> will contains the row index for those that have no value stored in the whole row
@param repeated_rows -> will contains the row index for those that have one unique value stored in the whole row
'''
for i in range(df.shape[0]):
    if len(df.iloc[i,1:].value_counts()) == 1:
        if str(df.iloc[i,1:].unique().tolist()[0]) == 'nan':
            empty_rows+=[i]
        else:            
            repeated_rows+=[i]

# Drop all rows with 100% missing/repeated values
df.drop(index=repeated_rows + empty_rows, inplace=True)

df.reset_index(drop=True, inplace=True)

In [8]:
# Check to ensure to more duplications exists (from blank rows)
print(df.duplicated().sum())

'''
Carry out numerous adjustments to the data set to ensure that it is converted into a readable and logical form
Initially we combine all values from the first few rows into one row in oredr to create a columns header row as one
Next we transpose/melt these columns and expand them onto every existing row to populate the dataset for each value.
'''
df_cols = df.loc[0:4].fillna('').apply('_-'.join).str.strip().tolist()
df.columns = df_cols
df_final = pd.melt(df.loc[5:], id_vars=['_-_-_-_-Date'])

# Split the column derived into numerous columns making it easier to visualize per each country, type, category etc.. 
cols_explode = ['Indicator','Index','Country','Type','Source']
df_final[cols_explode] = df_final['variable'].str.split('_-', expand=True)
df_final.columns = ['Date','variable','value']+cols_explode

0


In [9]:
# Filtering the countries necessary for this study
df_final = df_final[df_final['Country'].isin(['United States', 'Brazil', 'China', 'Germany', 'India', 'Japan', 'Mexico', 'United Kingdom'])]

df_rem_blank = df_final.groupby('variable').sum().reset_index()

df_final = df_final[df_final['variable'].isin(df_rem_blank[df_rem_blank['value']>0]['variable'].tolist())]

df_final.to_excel('data/conference_board_data_cleaned.xlsx', index=False)